In [7]:
import pandas as pd 
import sidetable as stb
import numpy as np

pd.options.display.max_columns = None

In [8]:
df = pd.read_csv('data/supermarket_sales - Sheet1.csv')
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [9]:
#Cambiamos el nombre de las columnas para unificarlas

df.rename(columns = {col: col.replace(' ', '_').lower() for col in df.columns}, inplace = True)

In [11]:
#Eliminamos la columna 'gross margin percentage' ya que nos da el mismo valor para todos los registros y no nos aporta información
#Eliminamos 'branch' porque es redundante con la columna city (cada supermercado corresponde a una ciudad).

df = df.drop(columns = ['invoice_id', 'branch', 'gross_margin_percentage'])
df.head(2)

,city,customer_type,gender,product_line,unit_price,quantity,tax_5%,total,date,time,payment,cogs,gross_income,rating
0,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,26.1415,9.1
1,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,3.8200,9.6


In [12]:
df.columns

Index(['city', 'customer_type', 'gender', 'product_line', 'unit_price',
       'quantity', 'tax_5%', 'total', 'date', 'time', 'payment', 'cogs',
       'gross_income', 'rating'],
      dtype='object')

In [13]:
df.shape

(1000, 14)

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
unit_price,1000.0,55.672130,26.494628,10.0800,32.875000,55.230,77.93500,99.96
quantity,1000.0,5.510000,2.923431,1.0000,3.000000,5.000,8.00000,10.00
tax_5%,1000.0,15.379369,11.708825,0.5085,5.924875,12.088,22.44525,49.65
total,1000.0,322.966749,245.885335,10.6785,124.422375,253.848,471.35025,1042.65
cogs,1000.0,307.587380,234.176510,10.1700,118.497500,241.760,448.90500,993.00
gross_income,1000.0,15.379369,11.708825,0.5085,5.924875,12.088,22.44525,49.65
rating,1000.0,6.972700,1.718580,4.0000,5.500000,7.000,8.50000,10.00


In [15]:
df.describe(include= "O").T

,count,unique,top,freq
city,1000,3,Yangon,340
customer_type,1000,2,Member,501
gender,1000,2,Female,501
product_line,1000,6,Fashion accessories,178
date,1000,89,2/7/2019,20
time,1000,506,19:48,7
payment,1000,3,Ewallet,345


In [16]:
# chequeamos tipo

df.dtypes

city              object
customer_type     object
gender            object
product_line      object
unit_price       float64
quantity           int64
tax_5%           float64
total            float64
date              object
time              object
payment           object
cogs             float64
gross_income     float64
rating           float64
dtype: object

In [17]:
# cambiamos la columna quantity a una categórica puesto que es falsa numérica

df["quantity"] = df["quantity"].astype("category")

In [18]:
# extraemos información sobre nulos

df.stb.missing()

,missing,total,percent
city,0,1000,0.0
customer_type,0,1000,0.0
gender,0,1000,0.0
product_line,0,1000,0.0
unit_price,0,1000,0.0
quantity,0,1000,0.0
tax_5%,0,1000,0.0
total,0,1000,0.0
date,0,1000,0.0
time,0,1000,0.0


In [19]:
# comprobar duplicados

df.duplicated().sum()

0

In [20]:
# df numéricas:

df_num = df.select_dtypes(include= ["float64", "int64"])
df_num.sample(3)

,unit_price,tax_5%,total,cogs,gross_income,rating
409,23.82,5.9550,125.0550,119.10,5.9550,5.4
55,85.98,34.3920,722.2320,687.84,34.3920,8.2
318,29.67,10.3845,218.0745,207.69,10.3845,8.1


In [21]:
# calcular outliers:

def detectar_outliers(lista_columnas, dataframe):
    dicc_indices = {} 
    
    for col in lista_columnas:
       
        Q1 = np.nanpercentile(dataframe[col], 25)
        Q3 = np.nanpercentile(dataframe[col], 75)
        
        IQR = Q3 - Q1
       
        outlier_step = 1.5 * IQR
 
        outliers_data = dataframe[(dataframe[col] < Q1 - outlier_step) | (dataframe[col] > Q3 + outlier_step)]
        if outliers_data.shape[0] > 0: 
            dicc_indices[col] = (list(outliers_data.index)) 
    return dicc_indices

In [22]:
lista_columnas = list(df_num.columns)

In [23]:
# Aplicamos función para que nos devuelva los índices

detectar_outliers(lista_columnas, df)

{'tax_5%': [166, 167, 350, 357, 422, 557, 699, 792, 996],
 'total': [166, 167, 350, 357, 422, 557, 699, 792, 996],
 'cogs': [166, 167, 350, 357, 422, 557, 699, 792, 996],
 'gross_income': [166, 167, 350, 357, 422, 557, 699, 792, 996]}

In [24]:
#Antes de continuar con nuestro proceso de ML agrupamos date y creamos columna nueva con los meses

df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
df['month'] = df['date'].dt.strftime('%B')

In [25]:
#Definimos una funcion para agrupar las horas por mañana o tarde. Antes de esto hemos comprobado que nuestros supermercados abren de 10 a 20.
def agrupar_turno(time):

    hora = int(time.split(':')[0])
    
    if 10 <= hora <= 14:
        return 'Morning'
    else:
        return 'Afternoon'

In [26]:
df['shift'] = df['time'].apply(agrupar_turno)

In [27]:
#Guardamos el dataframe modificado

df.to_pickle('data/supermarket_1.pkl')

### Estas son las explicaciones de las variables

    Branch: Branch of supercenter (3 branches are available identified by A, B and C).

    City: Location of supercenters

    Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.

    Gender: Gender type of customer
    
    Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel

    Unit price: Price of each product in $

    Quantity: Number of products purchased by customer

    Tax: 5% tax fee for customer buying

    Total: Total price including tax

    Date: Date of purchase (Record available from January 2019 to March 2019)

    Time: Purchase time (10am to 9pm)

    Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)

    COGS: Cost of goods sold

    Gross margin percentage: Gross margin percentage

    Gross income: Gross income

    Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)